In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [2]:
import os
from torch_geometric.datasets import TUDataset
import os.path as osp
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseSAGEConv, dense_diff_pool,dense_mincut_pool
import torch_geometric.transforms as T

import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_generation.load_graphs_and_create_metadata import dataset_metadata
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
import scipy.io as sco
import slam.io as sio
from scipy.special import softmax
import pickle
from scipy.stats import betabinom
import seaborn as sns
import tools.graph_processing as gp
import tools.graph_visu as gv
from matplotlib.pyplot import figure
import pandas as pd
import random
from torch_geometric.utils.convert import from_networkx
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch.nn.functional import one_hot
from sklearn.preprocessing import OneHotEncoder
from torch.nn import Linear
import torch.nn.functional as F
from math import ceil
import torch_geometric as pyg
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import TopKPooling
from torch_geometric.data import Data
from torch_geometric.loader import DenseDataLoader
from torch.nn.utils.rnn import pad_sequence
from torch_geometric.utils import to_dense_batch, to_dense_adj
from torch_geometric.nn import GCNConv, DenseGraphConv
%matplotlib inline

In [ ]:
path_trials = '..//data/simu_graph/simu_genders//'
path_sub_dir = '/noise_100,outliers_varied/graphs/'
all_trials = np.sort(list(map(int,os.listdir(path_trials)))) # all trials

In [ ]:
all_graphs_same_trial = []
all_graphs_labels_same_trial = []

referene_graphs = []

for trial in all_trials:
    
    referene_graphs.append(nx.read_gpickle(path_trials + str(trial) + '/reference_'+ str(trial) +'.gpickle'))
    
    trial_folder = path_trials + str(trial) + path_sub_dir
    all_graphs_same_trial.append(gp.load_graphs_in_list(trial_folder)) # append graphs
    all_graphs_labels_same_trial.append(np.ones([len(gp.load_graphs_in_list(trial_folder))])*trial) # append corr lab

In [ ]:
# taking 300 graph from each population for learning 

simu_train_set = []
simu_train_labels = []

for graph_set,graph_labels in zip(all_graphs_same_trial,all_graphs_labels_same_trial):
    simu_train_set.extend(graph_set[:300])
    simu_train_labels.extend(graph_labels[:300])

In [ ]:
all_graphs_same_trial[0][0].nodes[0]

In [ ]:
def graph_remove_dummy_nodes(graph):
    nodes_dummy_true = [x for x,y in graph.nodes(data=True) if y['is_dummy']==True]
    graph.remove_nodes_from(nodes_dummy_true)
    #print(len(graph.nodes))

In [ ]:
# Convert networkx graphs to pyg graphs

sulcal_simu_dataset = []

for i,g in enumerate(simu_train_set):
    #graph_remove_dummy_nodes(g) # remove dummy nodes
    g.remove_edges_from(nx.selfloop_edges(g)) # remove self loop edges
    
    attr_coords = np.array(list(nx.get_node_attributes(g,'coord').values())) #simu attribute (coords)
    
    x = torch.tensor(attr_coords,dtype=torch.float)
    
    #adj = torch.tensor(nx.adjacency_matrix(g).todense(),dtype=torch.float)
    
    y = torch.tensor(simu_train_labels[i],dtype=torch.long) # add graph label
    edge_index = torch.tensor(list(g.edges))
    
    sulcal_simu_dataset.append(Data(x=x, y=y, edge_index=edge_index.t().contiguous())) #

In [ ]:
len(sulcal_simu_dataset)

In [ ]:
random.shuffle(sulcal_simu_dataset)

train_dataset = sulcal_simu_dataset[:len(sulcal_simu_dataset)-500]
test_dataset = sulcal_simu_dataset[len(sulcal_simu_dataset)-500:]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # dense data loader for using adjacency
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
num_node_features = train_dataset[0].num_features
num_node_features

In [ ]:
class mincutnet(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=32):
        super(mincutnet, self).__init__()
        
        self.in_head = 1
        self.out_head = 1
        
        self.emb_dim = 16 # embedding dimension
        
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GATConv(hidden_channels, self.emb_dim, heads=self.in_head,dropout=0.2)

#        self.conv3 = GATConv(hidden_channels, self.emb_dim, concat=False, dropout=0.6)
        
        num_of_centers =  22
        self.pool1 = Linear(self.emb_dim, num_of_centers) # The degree of the node belonging to any of the centers
        
        self.conv3 = DenseGraphConv(self.emb_dim, self.emb_dim)

        #self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(self.emb_dim, out_channels)

    def forward(self, x, edge_index, batch): 
        
        x = F.relu(self.conv1(x, edge_index))
        #x = F.relu(self.conv2(x, edge_index))
        node_emb, attn_weights = self.conv2(x, edge_index,return_attention_weights=True)
        x = node_emb.relu()

        x, mask = to_dense_batch(x, batch) 
        
        adj = to_dense_adj(edge_index, batch) 
        s = self.pool1(x)

        
        x, adj, mincut_loss, ortho_loss = dense_mincut_pool(x, adj, s, mask) 
        x = self.conv3(x, adj) 
        g_emb = x.mean(dim=1) 
        #x = F.relu(self.lin1(x)) 
        out = self.lin2(g_emb)
        return out, g_emb, node_emb, s

In [ ]:
model = mincutnet(num_node_features, len(all_trials))
print(model)

In [ ]:
#model = GCN(hidden_channels=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    
    for data in train_loader:# Iterate in batches over the training dataset.
        out, g_emb, node_emb, s = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        out, g_emb, node_emb, s =  model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

In [ ]:
train_acc_lst = []
test_acc_lst = []
for epoch in range(1, 99):
    train()
    train_acc = test(train_loader)
    train_acc_lst.append(train_acc)
    test_acc = test(test_loader)
    test_acc_lst.append(test_acc)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
sns.set(rc={'figure.figsize':(11,6)})
epochs = np.arange(1,99,1)
plt.plot(epochs, train_acc_lst,label='Train accuracy')
plt.plot(epochs, test_acc_lst, label='Test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc="lower right")

### Evaluate on a seperate set

In [ ]:
# taking rest graph from each population as evaluation set

simu_eval_set = []
simu_eval_labels = []

for graph_set,graph_labels in zip(all_graphs_same_trial,all_graphs_labels_same_trial):
    simu_eval_set.extend(graph_set[300:])
    simu_eval_labels.extend(graph_labels[300:])

In [ ]:
eval_set = []

for i,g in enumerate(simu_eval_set):
    graph_remove_dummy_nodes(g) # remove dummy nodes
    g.remove_edges_from(nx.selfloop_edges(g)) # remove self loop edges
    
    attr_coords = np.array(list(nx.get_node_attributes(g,'coord').values())) #simu attribute (coords) 
    x = torch.tensor(attr_coords,dtype=torch.float)

    #x = torch.tensor(nx.adjacency_matrix(g).todense(),dtype=torch.float)
    
    y = torch.tensor(simu_eval_labels[i],dtype=torch.long) # add graph label
    edge_index = torch.tensor(list(g.edges))
    
    eval_set.append(Data(x=x, y=y, edge_index=edge_index.t().contiguous()))

In [ ]:
reference_set = []
reference_label = list(np.arange(0,10,1))

for i,g in enumerate(referene_graphs):
    #graph_remove_dummy_nodes(g) 
    g.remove_edges_from(nx.selfloop_edges(g)) 
    
    attr_coords = np.array(list(nx.get_node_attributes(g,'coord').values())) #simu attribute (coords) 
    x = torch.tensor(attr_coords,dtype=torch.float)
    
    y = torch.tensor(reference_label[i],dtype=torch.long) # add graph label
    edge_index = torch.tensor(list(g.edges))
    
    reference_set.append(Data(x=x, y=y, edge_index=edge_index.t().contiguous()))

In [ ]:
# eval emb mixed with reference graph
all_graph = DataLoader(eval_set+reference_set, batch_size= len(eval_set) + len(reference_set), shuffle=False)

for data in all_graph:
      out, g_emb, node_emb, s = model(data.x, data.edge_index, data.batch)
#   prob,g_emb,out_emb,attn_weights = model(data.x, data.edge_index, data.batch) o


#out_emb = out_emb.detach().numpy()
g_emb = g_emb.detach().numpy()

In [ ]:
def get_labels_from_k_means(k, coords):
    
    kmeans = KMeans(n_init= 'auto',n_clusters=k, random_state=0).fit(coords)
    
    return kmeans,kmeans.labels_

In [ ]:
k = 10      
kmeans,kmeans_labels = get_labels_from_k_means(k, g_emb[:2000])

In [ ]:
all_emb = np.concatenate((g_emb,kmeans.cluster_centers_),axis=0)

In [ ]:
from sklearn.manifold import TSNE
import pandas as pd

n_components = 2
tsne = TSNE(n_components)

tsne_graph = tsne.fit_transform(all_emb)
tsne_graph.shape

In [ ]:
sns.set(rc={'figure.figsize':(13,9)})

tsne_graph_df = pd.DataFrame({'tsne_1': tsne_graph[:2000,0], 'tsne_2': tsne_graph[:2000,1], 'label':simu_eval_labels})
tsne_ref_df = pd.DataFrame({'tsne_1': tsne_graph[2000:2010,0], 'tsne_2': tsne_graph[2000:2010,1], 'label':reference_label})
tsne_kmeans_df = pd.DataFrame({'tsne_1': tsne_graph[2010:,0], 'tsne_2': tsne_graph[2010:,1],'label':reference_label})

fig, ax = plt.subplots(1)
sns.scatterplot(x='tsne_1', y='tsne_2', hue='label', data=tsne_graph_df, ax=ax,s=20,palette='tab10')
sns.scatterplot(x='tsne_1', y='tsne_2', hue='label', data=tsne_ref_df, ax=ax,s=150,palette='tab10',markers='label')
plt.scatter(tsne_kmeans_df['tsne_1'], tsne_kmeans_df['tsne_2'], c='black', s = 100, marker='X')

# lim = (tsne_graph.min()-5, tsne_graph.max()+5)
# ax.set_xlim(lim)
# ax.set_ylim(lim)

ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

In [ ]:
from math import ceil

In [ ]:
ceil(0.25 * 80)

### Compare with classical methods on simulations

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(g_emb[:2000], simu_eval_labels, test_size=0.6, random_state=42)

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf_svm = SVC(gamma='auto',kernel='linear').fit(X_train, y_train)
clf_svm.score(X_test, y_test)

In [ ]:
clf_rf = RandomForestClassifier(max_depth = 5,random_state=0).fit(X_train, y_train)
clf_rf.score(X_test, y_test)

In [ ]:
# On coordinates

In [ ]:
avg_feature_coords = []

for d in eval_set:
    avg_feature_coords.append(np.mean(d.x.detach().numpy(),axis=0))
    
avg_feature_coords = np.array(avg_feature_coords)

In [ ]:
X_train_coords, X_test_coords, y_train_coords, y_test_coords = train_test_split(avg_feature_coords, simu_eval_labels, test_size=0.6, random_state=42)

In [ ]:
clf_coord = LogisticRegression(random_state=0).fit(X_train_coords, y_train_coords)
clf_coord.score(X_test_coords, y_test_coords)

In [ ]:
clf_svm_coord = SVC(gamma='auto',kernel='linear').fit(X_train_coords, y_train_coords)
clf_svm_coord.score(X_test_coords, y_test_coords)

In [ ]:
clf_rf_coord = RandomForestClassifier(max_depth = 5,random_state=0).fit(X_train_coords, y_train_coords)
clf_rf_coord.score(X_test_coords, y_test_coords)